# Convert RA and DEC to a Unit Direction in Heliocentric Frame

### Coordinates of Ceres According to JPL

In [1]:
# library imports
import numpy as np
import astropy
from astropy.coordinates import SkyCoord, ICRS, GCRS, HCRS, EarthLocation
from astropy.coordinates import HeliocentricMeanEcliptic, HeliocentricTrueEcliptic, BarycentricMeanEcliptic, BarycentricTrueEcliptic
from astropy.units import deg, au, km, meter, day, minute, second

#  MSE imports
import astro_utils
import asteroid_integrate
import asteroid_data
from asteroid_data import make_data_one_file, get_earth_pos
from astro_utils import qv2obs, qv2radec, qvrel2radec, report_radec_diff

In [2]:
# MJD of observation time
obstime_mjd = 58600.0

# Convert this to a JD
obstime_jd = astro_utils.mjd_to_jd(obstime_mjd)
print(f'Observation Time as JD: {obstime_jd}')

# Convert this to astropy time object
obstime = astropy.time.Time(obstime_mjd, format='mjd')
print(f'Observation Time in AP:', repr(obstime))

Observation Time as JD: 2458600.5
Observation Time in AP: <Time object: scale='utc' format='mjd' value=58600.0>


In [3]:
# Cartesian coordinates of Ceres in barycentric frame (ICRS)
q_ast_jpl_bary = [-1.358266736250873E+00, -2.365068287734591E+00, 1.753434815502372E-01] * au
v_ast_jpl_bary = [8.456717136811134E-03, -5.875569170947257E-03, -1.745541318980191E-03] * au / day

# Cartesian coordinates of Earth geocenter in barycentric frame
q_earth_jpl_bary = [-8.137850649885880E-01, -5.867610927308373E-01, -2.837047450366285E-06] * au
v_earth_jpl_bary = [ 9.867406393541108E-03, -1.395096307916507E-02,  1.404197076481182E-06] * au / day

# Cartesian coordinates of Sun in barycentric frame
q_sun_jpl_bary = [-1.699702116697901E-03, 7.591941485824569E-03, -3.294738263228371E-05] * au
v_sun_jpl_bary = [-8.347175817777104E-06, 7.503821640682956E-07,  2.164051447319061E-07] * au / day

# Compute heliocentric coordinates and velocities by subtracting out solar position / velocity
q_ast_jpl = q_ast_jpl_bary - q_sun_jpl_bary
v_ast_jpl = v_ast_jpl_bary - v_sun_jpl_bary
q_earth_jpl = q_earth_jpl_bary - q_sun_jpl_bary
v_earth_jpl = v_earth_jpl_bary - v_sun_jpl_bary

# Relative position and velocity from JPL
q_rel_jpl = q_ast_jpl - q_earth_jpl
v_rel_jpl = v_ast_jpl - v_earth_jpl

# Unit direction from JPL
u_jpl = q_rel_jpl.value / np.linalg.norm(q_rel_jpl.value)

In [4]:
# Relative coordinates and velocity of Ceres w.r.t. geocentric observer
# Same as above calculations, except quoted explicity by JPL
q_rel_jpl2 = [-5.444816712622853E-01, -1.778307195003754E+00,  1.753463185976876E-01] * au
v_rel_jpl2 = [-1.410689256729974E-03,  8.075393908217819E-03, -1.746945516056672E-03] * au / day

# q_rel_jpl2-q_rel_jpl
# v_rel_jpl2 - v_rel_jpl

# Range and range-rate according to JPL
rg_rel_jpl = 1.868042585592858E+00 * au
rg_rel_dot_jpl = 7.440278512685717E-03 * au / day

In [5]:
# Print position of Ceres, Earth, and relative according to JPL
print(f'Position (JPL):')
print(f'Earth:', q_earth_jpl)
print(f'Ceres:', q_ast_jpl)
print(f'Rel  :', q_rel_jpl)
print(f'Dir  :', u_jpl)

# Print relative velocity
print(f'\nRelative velocity (JPL):')
print(v_rel_jpl)

Position (JPL):
Earth: [-8.12085363e-01 -5.94353034e-01  3.01103352e-05] AU
Ceres: [-1.35656703 -2.37266023  0.17537643] AU
Rel  : [-0.54448167 -1.7783072   0.17534632] AU
Dir  : [-0.29147177 -0.95196288  0.09386634]

Relative velocity (JPL):
[-0.00141069  0.00807539 -0.00174695] AU / d


### MSE Integrated Coordinates of Ceres at MJD  58600

In [6]:
ast_elt = asteroid_integrate.load_data()

In [7]:
inputs, outputs = make_data_one_file(0, 1000)

In [8]:
ast_elt

,Num,Name,epoch_mjd,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
2,2,Pallas,58600.0,2.772466,0.230337,0.608007,3.020817,5.411373,1.041946,4.13,0.11,JPL 35,1.490912,1686.155979,0.003726,3.190951,3.639917,2.149005,-279.616804
3,3,Juno,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.20,0.32,JPL 34,-4.436417,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940421
5,5,Astraea,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541124,541124,2018 RP23,58600.0,2.586399,0.289358,0.088749,2.000720,3.913328,1.075531,17.30,0.15,JPL 7,1.654537,1519.293350,0.004136,0.706394,1.285400,-0.369137,-260.066715
541125,541125,2018 RV23,58600.0,3.113036,0.213678,0.203046,0.544794,0.242079,0.130760,16.10,0.15,JPL 8,0.206083,2006.201725,0.003132,0.917633,0.992956,0.786873,-41.751113
541126,541126,2018 RP24,58600.0,2.453880,0.176693,0.194504,2.649626,3.695880,0.937231,17.30,0.15,JPL 6,1.258854,1404.036362,0.004475,0.999551,1.321174,0.062320,-209.433076


In [9]:
# Index for Ceres
idx_ast = np.where(ast_elt.Name=='Ceres')

# Get the index for observation time
ts = inputs['ts'][idx_ast].flatten()

In [10]:
ts

array([51544., 51545., 51546., ..., 66517., 66518., 66519.], dtype=float32)

In [11]:
# Index of observation time <= obs_time
idx_t = np.searchsorted(ts, obstime_mjd-0.5)

In [12]:
t0 = ts[idx_t].item()
t1 = ts[idx_t+1].item()
w0 = (t1 - obstime_mjd) / (t1 - t0 )
w1 = (obstime_mjd - t0) / (t1 - t0)
print(f't0:      ', t0)
print(f't1:      ', t1)
print(f'obstime, ', obstime_mjd)
print(f'w0: ', w0)
print(f'w1: ', w1)

t0:       58600.0
t1:       58601.0
obstime,  58600.0
w0:  1.0
w1:  0.0


In [13]:
# Interpolated asteroid positions as observation time
q0 = outputs['q'][:, idx_t]
q1 = outputs['q'][:, idx_t+1]
q = w0 * q0 + w1 * q1

In [14]:
# Interpolated asteroid positions as observation time
v0 = outputs['v'][:, idx_t]
v1 = outputs['v'][:, idx_t+1]
v = w0 * v0 + w1 * v1

In [15]:
# Interpolated asteroid positions as observation time
u0 = outputs['u'][:, idx_t]
u1 = outputs['u'][:, idx_t+1]
u = w0 * u0 + w1 * u1

In [16]:
# Predicted position and direction of Ceres
q_ast_mse = q[idx_ast].flatten() * au

# Get position of Earth using utility function
q_earth_mse = get_earth_pos(obstime_mjd) * au

# Relative position
q_rel_mse = q_ast_mse - q_earth_mse

# Direction
u_mse = q_rel_mse.value / np.linalg.norm(q_rel_mse.value)

In [17]:
# Demonstrate that two methods of getting direction vector from MSE essentially identical

# Method 1: Normalize the manually computed displacement vector from earth to ast
u_mse2 = q_rel_mse.value / np.linalg.norm(q_rel_mse.value)

# Method 2: interpolate the pre-computed direction vector u
u_mse2 = u[idx_ast].flatten()
u_mse2 = u_mse2 / np.linalg.norm(u_mse2)
diff_sec = np.rad2deg(np.linalg.norm(u_mse - u_mse2))*3600
# Report
print(f'Difference between direction methods: ', f'{diff_sec:4.2e} arc seconds')

Difference between direction methods:  6.01e-03 arc seconds


In [18]:
# Print position of Ceres, Earth, and relative according to MSE
print(f'Position (MSE):')
print(f'Earth:', q_earth_mse)
print(f'Ceres:', q_ast_mse)
print(f'Rel  :', q_rel_mse)
print(f'Dir  :', u_mse)

# Print relative velocity
# print(f'\nRelative velocity (JPL):')
# print(v_rel_jpl)

Position (MSE):
Earth: [-8.12085390e-01 -5.94353020e-01  3.01103355e-05] AU
Ceres: [-1.3565673  -2.3726602   0.17537652] AU
Rel  : [-0.54448187 -1.77830714  0.17534641] AU
Dir  : [-0.29147187 -0.95196284  0.09386639]


### Compare JPL vs. MSE Integrated Coordinates

In [19]:
# Calculate error (MSE minus JPL) for position of Ceres, Earth, and relative
q_ast_err = q_ast_mse - q_ast_jpl
q_earth_err = q_earth_mse - q_earth_jpl
q_rel_err = q_rel_mse - q_rel_jpl
u_err = u_mse - u_jpl

# Compute vector norms
q_ast_err_norm = np.linalg.norm(q_ast_err.value)
q_earth_err_norm = np.linalg.norm(q_earth_err.value)
q_rel_err_norm = np.linalg.norm(q_rel_err.value)
u_err_norm = np.linalg.norm(u_err)
u_err_norm_sec = np.rad2deg(u_err_norm) * 3600

In [20]:
# Print position of Ceres, Earth, and relative according to MSE
print(f'Position Error (MSE-JPL):')
print(f'Ceres:', q_ast_err,  '; norm = ', f'{q_ast_err_norm:4.2e}')
print(f'Earth:', q_earth_err, '; norm = ', f'{q_earth_err_norm:4.2e}')
print(f'Rel  :', q_rel_err,  '; norm = ', f'{q_rel_err_norm:4.2e}')
print(f'Dir  :', u_err,  '; norm = ', f'{u_err_norm:4.2e} / {u_err_norm_sec:5.3f} arc seconds')

Position Error (MSE-JPL):
Ceres: [-2.29469036e-07  6.91557185e-08  9.06279444e-08] AU ; norm =  2.56e-07
Earth: [-2.72190521e-08  1.40254693e-08  3.19459110e-13] AU ; norm =  3.06e-08
Rel  : [-2.02249983e-07  5.51302493e-08  9.06276249e-08] AU ; norm =  2.28e-07
Dir  : [-1.05931825e-07  3.71436896e-08  4.77622761e-08] ; norm =  1.22e-07 / 0.025 arc seconds


**Conclusion**<br>
MSE results are all in heliocentric coordinates.<br>
Agreement to JPL coordinates is excellent, to order 1E-7 AU.<br>
Direction from earth 

### Observation of Ceres According to JPL

In [21]:
# Astrometric RA and DEC of Ceres at 2458600.5 according to JPL
ra = 252.250075738 * deg
dec = -17.009538673 * deg
delta = 1.86800464519883 * au
delta_dot = -12.8811808 * km / second

# Apparent RA and DEC
ra_app = 252.528732090 * deg
dec_app = -17.041522707 * deg

# Down-leg light time
light_time = 15.53572090 * minute

# Heliocentric coordinates of Ceres
ast_hel_lon = 240.239114 * deg
ast_hel_lat = 3.671885 * deg
ast_r = 2.738704926 * au
ast_r_dot = 1.3611037 * km / second

# Heliocentric coordinates of Earth
# earth_hel_lon_deg = 216.1950 * deg
# earth_hel_lat_deg = 0.0017 * deg
# earth_r_au = 1.006347536397

In [22]:
# Reference frames
# https://docs.astropy.org/en/stable/api/astropy.coordinates.ICRS.html
frame_solar = HeliocentricMeanEcliptic
frame_earth = ICRS

In [23]:
# Earth location of the Palomar observatory
geoloc_palomar_auto = EarthLocation.of_site('Palomar')
print(f'geolocation of Palomar observatory:')
print(f'geoloc_palomar = {geoloc_palomar_auto}')
print(f'geoloc_palomar geodetic = {geoloc_palomar_auto.geodetic}')

geolocation of Palomar observatory:
geoloc_palomar = (-2410346.78217658, -4758666.82504051, 3487942.97502457) m
geoloc_palomar geodetic = GeodeticLocation(lon=<Longitude -116.863 deg>, lat=<Latitude 33.356 deg>, height=<Quantity 1706. m>)


In [24]:
# For some reason, the 'nice' way to do this below leads to a bug in constructor to GCRS
# geoloc_palomar = EarthLocation(-2410346.8, -4758666.8, 3487943, unit='m')

# Workaround: extract the geolocation of the Palomar observatory using EarthLocation.of_site
geoloc_palomar = [-2410346.78217658, -4758666.82504051, 3487942.97502457] * meter

xxx

In [ ]:
# Extract number part of JPL RA and DEC
ra_jpl, dec_jpl, r_jpl = ra.value, dec.value, delta.value

In [ ]:
dq_lt = v_earth_jpl * light_time.to(day)
dq_lt

In [ ]:
# Calculated RA, DEC
ra_, dec_, r_ = qv2radec(q=q_ast_jpl_bary, v=v_ast_jpl_bary, mjd=obstime_mjd, 
                         frame=BarycentricMeanEcliptic, light_lag=False)

# Report results
print(f'Calculations using qv2radec() with JPL positions as inputs, no light lag.')
err=report_radec_diff(name1='JPL', name2='Calc', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_, dec2=dec_, obstime_mjd=obstime_mjd)

In [ ]:
# Calculated RA, DEC
ra_, dec_, r_ = qv2radec(q=q_ast_jpl_bary, v=v_ast_jpl_bary, mjd=obstime_mjd, 
                         frame=BarycentricMeanEcliptic, light_lag=True)

# Report results
print(f'Calculations using qv2radec() with JPL positions as inputs, with light lag.')
err=report_radec_diff(name1='JPL', name2='Calc', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_, dec2=dec_, obstime_mjd=obstime_mjd)

In [ ]:
obs.distance

In [ ]:
light_time = obs.distance.to(meter) / light_speed
light_time

In [ ]:
v_ast_jpl.to(au/day) * light_time.to(day)

In [ ]:
dir(obs)

In [ ]:
# Calculated RA, DEC
ra_, dec_, r_ = qvrel2radec(q_body=q_ast_jpl_bary-dq_lt*1.0, v_body=v_ast_jpl_bary,
                            q_earth=q_earth_jpl_bary, v_earth=v_earth_jpl_bary,
                            mjd=obstime_mjd, frame=BarycentricMeanEcliptic)

# Report results
print(f'Calculations using qvrel2radec() with JPL positions as inputs')
report_radec_diff(name1='JPL', name2='Calc', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_, dec2=dec_, obstime_mjd=obstime_mjd)

### JPL Calculates RA/DEC Differently Than Astropy!

In [ ]:
# Set the frames for solar and earth
# frame_solar = HeliocentricMeanEcliptic
# frame_earth = ICRS

# Create the observation in the Solar frame using Cartesian coordinates equal to the JPL displacement
x, y, z = q_rel_jpl
v_x, v_y, v_z = v_rel_jpl
obs_solar = SkyCoord(x=x, y=y, z=z, v_x=v_x, v_y=v_y, v_z=v_z, obstime=obstime, 
                     representation_type='cartesian', frame=frame_solar)

# Get the RA and DEC in the earth frame (ICRS) from astropy.  This is not the same as the way JPL does it!
obs_earth = obs_solar.transform_to(frame_earth)

# Unpack the RA, DEC and distance from astropy
ra_ap, dec_ap, delta_ap = obs_earth.ra, obs_earth.dec, obs_earth.distance

# Display the observed position as an AP SkyCoord in spherical coordinates
print('Solar Frame:', obs_solar)
print('Earth Frame:', obs_earth)

print(f'\nAP Spherical Coordinates (diff to JPL):')
print(f'RA : {ra_ap:10.6f} ({ra-ra_ap:10.6f})')
print(f'DEC: {dec_ap:10.6f} ({dec-dec_ap:10.6f})')
print(f'R  : {delta_ap:10.6f} ({delta-delta_ap:10.6f})')

In [ ]:
obs_solar

In [ ]:
v_rel_jpl

In [ ]:
# Demonstrate that astropy is internally consistent.
# If we convert the RA and DEC back to a Cartesian representation, we recover the input position

# Create an observation from the RA and DEC extracted from the Cartesian representation
obs_earth_rec = SkyCoord(ra=ra_ap, dec=dec_ap, distance=delta_ap, obstime=obstime, frame=frame_earth)

# Transform it back to Cartesian coordinates in the Heliocentric frame
obs_solar_rec = obs_earth_rec.transform_to(frame_solar)

# Extract its Cartesian coordinates
q_rel_ap = obs_solar_rec.cartesian.xyz

# Compare to the starting coordinates: they are very close!
q_err = q_rel_ap - q_rel_jpl
q_err_norm = np.linalg.norm(q_err.value)

# Report results
print('Displacement:')
print('JPL:', q_rel_jpl)
print('AP :', q_rel_ap)

print(f'\nError in recovery of q_rel from round trip in astropy:')
print(f'q_err:', q_err)
print(f'Norm:  {q_err_norm:4.2e}')

**Conclusion**<br>
A round trip between the solar frame (HeliocentricMeanEcliptic) and earth frame (ICRS) is internally consistant in astropy up to float precision.<br>
Error is negligible, on the order of $10^{-15}$ AU.<br>
The astropy RA and DEC are slightly but meaningfully different from the JPL calculations.

In [ ]:
# Demonstrate that astropy gets different values when fed the RA and DEC from JPL

# Create an observation from the RA and DEC quoted by JPL
obs_earth_jpl = SkyCoord(ra=ra, dec=dec, distance=delta, obstime=obstime, frame=frame_earth)

# Convert this to the earth frame using the same exact method as above
obs_solar_jpl = obs_earth_jpl.transform_to(frame_solar)

# Extract its Cartesian coordinates
q_rel_jpl2 = obs_solar_jpl.cartesian.xyz

# Get the implied directions
u_jpl = q_rel_jpl.value / np.linalg.norm(q_rel_jpl.value)
u_jpl2 = q_rel_jpl2.value / np.linalg.norm(q_rel_jpl2.value)

# Compare to the starting coordinates: they are very close!
q_err = q_rel_jpl2 - q_rel_jpl
q_err_norm = np.linalg.norm(q_err.value)

# Error in direction
u_err = u_jpl2 - u_jpl
u_err_norm = np.linalg.norm(u_err)
u_err_deg = np.rad2deg(u_err_norm)

# Report results
print('Displacement:')
print('JPL from Vectors:', q_rel_jpl)
print('JPL from RA, DEC:', q_rel_jpl2)

print(f'\nError in recovery of q_rel from RA and DEC quoted by JPL:')
print(f'q_err:', q_err)
print(f'Norm:  {q_err_norm:4.2e}')

print(f'\nError in recovery of u from RA and DEC quoted by JPL:')
print(f'u_err:', u_err)
print(f'Norm:  {u_err_norm:4.2e} ({u_err_deg:8.6f} degrees)')

### Try Simple Light-Time Adustment

In [ ]:
q_rel_jpl

In [ ]:
light_time

In [ ]:
light_time.to(day)

In [ ]:
v_rel_jpl

In [ ]:
dq = v_earth_jpl * light_time.to(day)
dq

In [ ]:
q_rel_adj - q_rel_jpl

In [ ]:
# Create the observation in the Solar frame using Cartesian coordinates equal to the JPL displacement
dq = v_ast_jpl * light_time.to(day)
q_rel_adj = q_rel_jpl + dq
x, y, z = q_rel_adj
v_x, v_y, v_z = v_rel_jpl
# q_rel_adj = 
obs_solar = SkyCoord(x=x, y=y, z=z, v_x=v_x, v_y=v_y, v_z=v_z, obstime=obstime, 
                     representation_type='cartesian', frame=frame_solar)

# Get the RA and DEC in the earth frame (ICRS) from astropy.  This is not the same as the way JPL does it!
obs_earth = obs_solar.transform_to(frame_earth)

# Unpack the RA, DEC and distance from astropy
ra_ap, dec_ap, delta_ap = obs_earth.ra, obs_earth.dec, obs_earth.distance

# Display the observed position as an AP SkyCoord in spherical coordinates
print('Solar Frame:', obs_solar)
print('Earth Frame:', obs_earth)

print(f'\nJPL Spherical Coordinates (diff):')
print(f'RA : {ra_ap:10.6f} ({ra-ra_ap:10.6f})')
print(f'DEC: {dec_ap:10.6f} ({dec-dec_ap:10.6f})')
print(f'R  : {delta_ap:10.6f} ({delta-delta_ap:10.6f})')

**Results without Light Time Adjustment**<br>
```
AP Spherical Coordinates (diff to JPL):
RA : 252.132890 deg (  0.117186 deg)
DEC: -16.977729 deg ( -0.031810 deg)
R  :   1.861312 AU (  0.006693 AU
```

In [ ]:
q_rel_jpl.value

In [ ]:
q_rel_jpl

### Coordinates of Ceres at MJD 58600 as SkyCoord instance

In [ ]:
# Alternate syntaxes for creating a SkyCoord class
# c = SkyCoord(ra=ra_deg, dec=dec_deg, unit='deg', frame=frame_earth)
# c = SkyCoord(ra=ra_deg*astropy.units.deg, dec=dec_deg*astropy.units.deg, distance=delta_au*astropy.units.au, frame=frame_earth)
# c = SkyCoord(ra=ra_deg, dec=dec_deg, distance=delta_au, unit='deg', frame=frame_earth)

In [ ]:
# Distances of zero and one AU
zero_au = 0.0 * au 
one_au = 1.0 * au

# Angle of zero degrees
zero_deg = 0.0 * deg

# Observation time
obstime = astropy.time.Time(val=obstime_mjd, format='mjd')

# Display class outputs
print(f'Observation of Ceres according to JPL:')
print(f'obstime  = {repr(obstime)}')
print(f'ra       = {ra}')
print(f'dec      = {dec}')
print(f'delta    = {delta}')
print(f'ast r    = {ast_r}')

In [ ]:
# Coordinates of Ceres in geocentric frame; include observation time & location.  
# Use correct distance delta_au from JPL
# ast_geo = SkyCoord(ra=ra, dec=dec, distance=distance, obstime=obstime, frame=GCRS, obsgeoloc=geoloc_palomar)
ast_geo = SkyCoord(ra=ra, dec=dec, distance=delta, obstime=obstime, frame=frame_earth)
ast_geo

In [ ]:
# Coordinates of Earth in geocentric frame; this is easy, distance is zero!
# earth_geo = GCRS(ra=zero_deg, dec=zero_deg, distance=zero_au, obstime=obstime, obsgeoloc = geoloc_palomar)
earth_geo = SkyCoord(ra=zero_deg, dec=zero_deg, distance=zero_au, obstime=obstime, frame=frame_earth)
earth_geo

### Unit direction of Ceres in Heliocentric (J2000) Frame from RA and Dec

### Old version

In [ ]:
ast_gcrs = SkyCoord(ra=ra, dec=dec, distance=delta, obstime=obstime, frame=GCRS)
earth_gcrs = SkyCoord(ra=zero_deg, dec=zero_deg, distance=zero_au, obstime=obstime, frame=GCRS)

# convert to BarycentricMeanEcliptic
ast_bme = ast_gcrs.transform_to(BarycentricMeanEcliptic)
earth_bme = earth_gcrs.transform_to(BarycentricMeanEcliptic)

In [ ]:
ast_bme.cartesian.xyz

In [ ]:
q_ast_jpl

In [ ]:
# The relative displacement
rel_bme = ast_bme.cartesian - earth_bme.cartesian

# Compare coordinates
q_err_ast = ast_bme.cartesian.xyz - q_ast_jpl
q_err_earth = earth_bme.cartesian.xyz - q_earth_jpl
q_err_rel = rel_bme.xyz - q_rel_jpl

# Norms of errors
q_err_ast_norm = np.linalg.norm(q_err_ast.value)
q_err_earth_norm = np.linalg.norm(q_err_earth.value)
q_err_rel_norm = np.linalg.norm(q_err_rel.value)

print(f'Displacement error:')
print(f'Ceres: ', q_err_ast)
print(f'Earth: ', q_err_earth)
print(f'Rel  : ', q_err_rel)
print(f'\nNorm of errors:')
print(f'Ceres: {q_err_ast_norm:4.2e}')
print(f'Earth: {q_err_earth_norm:4.2e}')
print(f'Rel  : {q_err_rel_norm:4.2e}')

In [ ]:
# set the two frames
frame_earth = GCRS
# frame_solar = BarycentricMeanEcliptic
# frame_solar = BarycentricTrueEcliptic
frame_solar = HeliocentricMeanEcliptic
# frame_solar = HeliocentricTrueEcliptic

# position of the asteroid and earth in the earth frame
ast_geo = SkyCoord(ra=ra, dec=dec, distance=delta, obstime=obstime, frame=frame_earth)
earth_geo = SkyCoord(ra=zero_deg, dec=zero_deg, distance=zero_au, obstime=obstime, frame=frame_earth)

# convert to solar frame
ast_solar = ast_geo.transform_to(frame_solar)
earth_solar = earth_geo.transform_to(frame_solar)

In [ ]:
# The relative displacement
rel_solar = ast_solar.cartesian - earth_solar.cartesian

# Compare coordinates
q_err_ast = ast_solar.cartesian.xyz - q_ast_jpl
q_err_earth = earth_solar.cartesian.xyz - q_earth_jpl
q_err_rel = rel_solar.xyz - q_rel_jpl

# Norms of errors
q_err_ast_norm = np.linalg.norm(q_err_ast.value)
q_err_earth_norm = np.linalg.norm(q_err_earth.value)
q_err_rel_norm = np.linalg.norm(q_err_rel.value)

print(f'Displacement error:')
print(f'Ceres: ', q_err_ast)
print(f'Earth: ', q_err_earth)
print(f'Rel  : ', q_err_rel)
print(f'\nNorm of errors:')
print(f'Ceres: {q_err_ast_norm:4.2e}')
print(f'Earth: {q_err_earth_norm:4.2e}')
print(f'Rel  : {q_err_rel_norm:4.2e}')

In [ ]:
SkyCoord(ra=ra, dec=dec, distance=delta, obstime=obstime, frame=GCRS).cartesian

In [ ]:
SkyCoord(ra=ra, dec=dec, distance=delta, obstime=obstime, frame=ICRS).cartesian

In [ ]:
x, y, z = q_rel_jpl
obs = SkyCoord(x=x, y=y, z=z, representation_type='cartesian', obstime=obstime, frame=ICRS)

In [ ]:
# recovered displacement
q_rel_rec = obs.represent_as('cartesian')
q_rel_rec

In [ ]:
q_rel_rec.xyz - q_rel_jpl

In [ ]:
# Relative displacement from Earth to Ceres in the Heliocentric frame
# This is critical, need to transform from GCRS to Heliocentric for axes to be oriented correctly
ast_hel = ast_geo.transform_to(frame_solar)
earth_hel = earth_geo.transform_to(frame_solar)
rel_hel = (ast_hel.cartesian - earth_hel.cartesian)
print(f'Heliocentric Cartesian coordinates from JPL RA / DEC Data:')
print('Ceres:   ', ast_hel.cartesian)
print('Earth:   ', earth_hel.cartesian)
print('Relative:', rel_hel)

In [ ]:
# Extract arrays from heliocentric objects
ast_jpl = ast_hel.cartesian.xyz.value
earth_jpl = earth_hel.cartesian.xyz.value
rel_jpl = rel_hel.xyz.value

In [ ]:
# Direction from earth to asteroid
u_jpl = rel_jpl / np.linalg.norm(rel_jpl)
print('Direction from Earth to Ceres:')
np.round(u_jpl, 6)

In [ ]:
# Calculation in a single step using coordinate transformations.
# Don't need the distance, just the RA and the Dec!
# ast_geo_hcrs = SkyCoord(ra=ra, dec=dec, distance=one_au, obstime=obstime, frame=HCRS)
rel_geo_hcrs = SkyCoord(ra=ra, dec=dec, distance=delta, obstime=obstime, frame=HCRS)
u_jpl_hcrs = rel_geo_hcrs.transform_to(frame_solar).cartesian.xyz.value
u_jpl_hcrs = u_jpl_hcrs / np.linalg.norm(u_jpl_hcrs)
norm_diff = np.linalg.norm(u_jpl - u_jpl_hcrs)
print(f'Difference between u_jpl by subtraction and method with HCRS frame:')
print(f'Norm of difference =  {norm_diff:8.6f} = {np.rad2deg(norm_diff):5.3f} degrees')

In [ ]:
# Demonstration that alternate method of subtracting in HCRS frame is exactly the same as one line transform
# delta_hel_from_hcrs = SkyCoord
ast_hel_from_hcrs = SkyCoord(ra=ra, dec=dec, distance=delta, obstime=obstime, frame=HCRS).transform_to(frame_solar)
earth_hel_from_hcrs = SkyCoord(ra=zero_deg, dec=zero_deg, distance=zero_au, obstime=obstime, frame=HCRS).transform_to(frame_solar)
rel_jpl_hcrs2 = (ast_hel_from_hcrs.cartesian.xyz.value - earth_hel_from_hcrs.cartesian.xyz.value)
u_jpl_hcrs2 = rel_jpl_hcrs2 / np.linalg.norm(rel_jpl_hcrs2)
norm_diff2 = np.linalg.norm(u_jpl - u_jpl_hcrs2)
print(f'Difference between u_jpl by subtraction and method with HCRS frame:')
print(f'Norm of difference =  {norm_diff2:8.6f} = {np.rad2deg(norm_diff2):5.3f} degrees')

In [ ]:
rel_jpl_hcrs2

### Conclusion: Conversion Procedure
We can convert from an RA and DEC to a unit direction in two lines of code by:<br>
1) Create the initial observation as a SkyCoord instance in the **HCRS frame** (Heloiocentric, axis aligned to GCRS)<br>
2) Convert this observation to the **HeliocentricMeanEcliptic frame**<br>
The two coordinate systems share the same origin (the sun, NOT the solar system barycenter).<br>
The only difference is the axis alignment; the axes in the HCRS are aligned with earth's equator / north pole, while the axes in HeliocentricMeanEcliptic are have the z-axis orthogonal to the ecliptic, with the x-axis pointing to the equinox.<br>
Experiments show that the same procedure using either ICRS or GCRS are not consistent with the explicit two step procedure.

### Compare two methods of Using JPL Data to Generate Directions

In [ ]:
# Compare two methods 
diff_ast = ast_jpl2 - ast_jpl
diff_earth = earth_jpl2 - earth_jpl
diff_rel = rel_jpl2 - rel_jpl
diff_u = u_jpl2 - u_jpl
diff_u_norm = np.linalg.norm(diff_u)

print(f'Difference in position (heliocentric long/lat - RA/DEC)')
print(f'Asteroid  : norm={np.linalg.norm(diff_ast):5.2e} AU')
print(f'Earth     : norm={np.linalg.norm(diff_earth):5.2e} AU')
print(f'Rel       : norm={np.linalg.norm(diff_rel):5.2e} AU')
print(f'Direction : norm={diff_u_norm:5.2e} / {np.rad2deg(diff_u_norm):6.4f} degrees')
print(f'Asteroid: ', diff_ast)
print('Earth    :', diff_earth)
print('Rel      :', diff_rel)

In [ ]:
# MSE direction via lookup of output field u
u0 = outputs['u'][:, idx_t]
u1 = outputs['u'][:, idx_t+1]
u = w0 * u0 + w1 * u1
u_mse = u[idx_ast].flatten()
np.round(u_mse, 6)

In [ ]:
# Demonstrate that two methods of getting direction vector from MSE essentially identical
np.linalg.norm(u_mse - u_mse2)

In [ ]:
print(f'Heliocentric Cartesian coordinates from MSE Integration:')
print('Ceres:   ', ast_mse)
print('Earth:   ', earth_mse)
print('Relative:', rel_mse)
print('Direction:', u_mse)

**Conclusion**<br>
We can look up the calculated direction from Earth to an asteroid in one line by:<br>
1) Load the asteroid direction data by calling <br>
```inputs, outputs = make_data_one_file(0, 1000)```<br>
2) Extract the times as ```inputs['ts'][idx_ast]``` where ```idx_ast``` is the index of the desired asteroid<br>
3) Extract the computed directions as ```outputs['u'][idx_ast]```<br>
4) Compute predicted direction by interpolating the desired time (as an MJD)<br>
These results are consistent with manually subtracting the integrated asteroid position minus the earth's position in the heliocentric frame.

### Compare MSE Calculations to JPL Using RA / DEC

In [ ]:
ast_err = ast_mse - ast_jpl
ast_err_norm = np.linalg.norm(ast_err)
print('ast_err: ', ast_err)
print(f'norm ast_err = {ast_err_norm:5.2e}')

In [ ]:
earth_err = earth_mse - earth_jpl
earth_err_norm = np.linalg.norm(earth_err)
print('earth_err: ', earth_err)
print(f'norm earth_err = {earth_err_norm:5.2e}')

In [ ]:
rel_err = rel_mse - rel_jpl
rel_err_norm = np.linalg.norm(rel_err)
print('rel_err: ', rel_err)
print(f'norm rel_err = {rel_err_norm:5.2e}')

In [ ]:
u_err = u_mse - u_jpl
u_err_norm = np.linalg.norm(u_err)
print('direction_err: ', u_err)
print(f'norm direction_err = {u_err_norm:5.2e}')
print(f'in degrees = {np.rad2deg(u_err_norm):6.4f} = ({np.rad2deg(u_err_norm)*3600:4.1f} seconds)')

**Conclusion**<br>
Directions predicted by MSE integration are consistent with results from JPL to within 0.0079 degrees / 28 seconds.<br>